In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Subset

In [2]:
#data loader adopted from training
def data_loader(data_dir,
                    batch_size,
                    random_seed=42,
                    valid_size=0.1,
                    shuffle=True,
                    test=False):

        normalize = transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010],
        )

        # define transforms
        transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
        download = not os.path.exists(os.path.join(data_dir, "cifar-10-batches-py"))




        if test:
          dataset = datasets.CIFAR10(
            root=data_dir, train=False,
            download=download, transform=transform,
          )
          indices = list(range(0, len(dataset)))
          np.random.seed(42)
          np.random.shuffle(indices)
          data = Subset(dataset,indices)
        else:
          dataset = datasets.CIFAR10(
            root=data_dir, train=True,
            download=download, transform=transform,
          )
          indices = list(range(0, len(dataset)))
          np.random.seed(42)
          np.random.shuffle(indices)
          data = Subset(dataset,indices)

        data_loader = torch.utils.data.DataLoader(
            data, batch_size=batch_size, shuffle=shuffle
        )



        return data_loader


calib_loader = data_loader(data_dir='/content/rn',
                                  batch_size=32,
                                  test=False)
test_loader = data_loader(data_dir='/content/rn',
                                  batch_size=32,
                                  test=True)

100%|██████████| 170M/170M [00:02<00:00, 63.5MB/s]


Extracting /content/rn/cifar-10-python.tar.gz to /content/rn


## We load the original Model

In [7]:
import torch
import torch.nn as nn

class VGG19(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG19, self).__init__()
        self.in_channels = 3

        # Convolutional layers (atomic definition without Sequential)
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_4 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_4 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_4 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(512, 4096)
        self.relu_fc1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(4096, 4096)
        self.relu_fc2 = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(4096, num_classes)

        # Weight initialization
        self._initialize_weights()

    def forward(self, x):
        x = self.relu1_1(self.conv1_1(x))
        x = self.relu1_2(self.conv1_2(x))
        x = self.pool1(x)

        x = self.relu2_1(self.conv2_1(x))
        x = self.relu2_2(self.conv2_2(x))
        x = self.pool2(x)

        x = self.relu3_1(self.conv3_1(x))
        x = self.relu3_2(self.conv3_2(x))
        x = self.relu3_3(self.conv3_3(x))
        x = self.relu3_4(self.conv3_4(x))
        x = self.pool3(x)

        x = self.relu4_1(self.conv4_1(x))
        x = self.relu4_2(self.conv4_2(x))
        x = self.relu4_3(self.conv4_3(x))
        x = self.relu4_4(self.conv4_4(x))
        x = self.pool4(x)

        x = self.relu5_1(self.conv5_1(x))
        x = self.relu5_2(self.conv5_2(x))
        x = self.relu5_3(self.conv5_3(x))
        x = self.relu5_4(self.conv5_4(x))
        x = self.pool5(x)

        x = x.view(x.size(0), -1)
        x = self.relu_fc1(self.fc1(x))
        x = self.relu_fc2(self.fc2(x))
        x = self.fc3(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


## We then edit the original model to be suitable for quantization

In [8]:
import torch
import torch.nn as nn

class QuantVGG19(nn.Module):
    def __init__(self, num_classes=10):
        super(QuantVGG19, self).__init__()
        self.in_channels = 3
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

        # Convolutional layers (atomic definition without Sequential)
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_4 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_4 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_4 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(512, 4096)
        self.relu_fc1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(4096, 4096)
        self.relu_fc2 = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(4096, num_classes)

        # Weight initialization
        self._initialize_weights()

    def forward(self, x):
        x = self.quant(x)
        x = self.relu1_1(self.conv1_1(x))
        x = self.relu1_2(self.conv1_2(x))
        x = self.pool1(x)

        x = self.relu2_1(self.conv2_1(x))
        x = self.relu2_2(self.conv2_2(x))
        x = self.pool2(x)

        x = self.relu3_1(self.conv3_1(x))
        x = self.relu3_2(self.conv3_2(x))
        x = self.relu3_3(self.conv3_3(x))
        x = self.relu3_4(self.conv3_4(x))
        x = self.pool3(x)

        x = self.relu4_1(self.conv4_1(x))
        x = self.relu4_2(self.conv4_2(x))
        x = self.relu4_3(self.conv4_3(x))
        x = self.relu4_4(self.conv4_4(x))
        x = self.pool4(x)

        x = self.relu5_1(self.conv5_1(x))
        x = self.relu5_2(self.conv5_2(x))
        x = self.relu5_3(self.conv5_3(x))
        x = self.relu5_4(self.conv5_4(x))
        x = self.pool5(x)

        x = x.view(x.size(0), -1)
        x = self.relu_fc1(self.fc1(x))
        x = self.relu_fc2(self.fc2(x))
        x = self.fc3(x)
        x = self.dequant(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


We load the original trained models into the updated model architecture.

we then add observers to see how our calibration data is.

In [ ]:
model = QuantVGG19()
model.eval()
checkpoint = torch.load("/content/vgg19_checkpoint3_9.pth", map_location="cpu")
model.load_state_dict(checkpoint['model_state'])

#model.qconfig = torch.quantization.get_default_qconfig("fbgemm")
#model.qconfig = torch.quantization.default_qconfig
model.qconfig = torch.quantization.QConfig(
    activation=torch.quantization.MinMaxObserver.with_args(dtype=torch.quint8),
    weight=torch.quantization.MinMaxObserver.with_args(dtype=torch.qint8)
)

modules = [
    ['conv1_1', 'relu1_1'],
    ['conv1_2', 'relu1_2'],
    ['conv2_1', 'relu2_1'],
    ['conv2_2', 'relu2_2'],
    ['conv3_1', 'relu3_1'],
    ['conv3_2', 'relu3_2'],
    ['conv3_3', 'relu3_3'],
    ['conv3_4', 'relu3_4'],
    ['conv4_1', 'relu4_1'],
    ['conv4_2', 'relu4_2'],
    ['conv4_3', 'relu4_3'],
    ['conv4_4', 'relu4_4'],
    ['conv5_1', 'relu5_1'],
    ['conv5_2', 'relu5_2'],
    ['conv5_3', 'relu5_3'],
    ['conv5_4', 'relu5_4'],
    ['fc1', 'relu_fc1'],
    ['fc2', 'relu_fc2']
]


model = torch.quantization.fuse_modules(model, modules)
model.eval()
model_prepared = torch.quantization.prepare(model)
print("Number of observers:", len([m for m in model_prepared.modules() if 'Observer' in str(type(m))]))
model_prepared.eval()
with torch.no_grad():
    for batch, _ in calib_loader:
        batch = batch.to('cpu')
        model_prepared(batch)

quantized_model = torch.quantization.convert(model_prepared)


We load the original model for comparison

In [ ]:
modelfp32 = VGG19()
checkpoint = torch.load("/content/vgg19_checkpoint3_9.pth", map_location="cpu")
modelfp32.load_state_dict(checkpoint['model_state'])

In [13]:
import time
total=0
correct=0
total_time = 0
quantized_model.eval()
with torch.no_grad():
    for data in test_loader:

        inputs, labels = data
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        t0 = time.time()
        outputs = quantized_model(inputs)
        t1 = time.time()
        total_time += (t1-t0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
acc = correct/total
t_av = total_time/total #average inference time per image
print("Quantized ACC: ", acc*100, "Average Inference on Quantized: ", t_av)

Quantized ACC:  70.73 Average Inference on Quantized:  0.008483899807929993


In [14]:
import time
total=0
correct=0
total_time = 0
modelfp32.eval()
with torch.no_grad():
    for data in test_loader:

        inputs, labels = data
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        t0 = time.time()
        outputs = modelfp32(inputs)
        t1 = time.time()
        total_time += (t1-t0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
acc = correct/total
t_av = total_time/total #average inference time per image
print("fp32 ACC: ", acc*100, "Average Inference on fp32: ", t_av)

fp32 ACC:  71.11 Average Inference on fp32:  0.018399335145950317


# Our Scores

Accuracy Retention = (Quantized Accuracy / Original Accuracy) × 100
                   = *99.46%*

Inference Time=( Original Inference Time/ Original Inference Time)
              = *2.168x*

Wow, We get amazing inference improvement in VGG19!!!!

Overall = Accuracy Retention/100 + Inference Speedup/5 = 1.4266 
 
Lets look at the others!!